In [1]:
import pymCADRE
import pandas as pd
import numpy as np
import gurobipy
from pymCADRE.rank.rank_reactions import *
from pymCADRE.prune.prune_model import *
import os


In [2]:
os.environ['GUROBI_HOME'] = "/home/jiang817/cybergut_project/Software/gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "/home/jiang817/cybergut_project/Software/gurobi_license/gurobi.lic"

In [3]:
model = io.read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 921650
Academic license - for non-commercial use only - registered to jiang817@purdue.edu


In [4]:
for i in range (len(model.reactions)):
    if model.reactions[i].id.startswith('EX_'):
        model.reactions[i].bounds = (-100, 100)

In [5]:
G = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_pymCADRE_INPUT/ENTRZE_ID_RECON3D_XML.csv')
G = list(G['ENTREZ_GENE_ID'])


In [6]:
U = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_pymCADRE_INPUT/Ubiquity_Score_RECON3D_XML.csv')
U = list(U['Ubiquity_score'])


In [7]:
confidence_scores = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_pymCADRE_INPUT/Recon3D_XML_confidence_scores.csv')
confidence_scores = np.float64(list(confidence_scores['Confidence Score']))

In [ ]:
#protected_rxn = ['BIOMASS_maintenance', 'ATPS4mi']
#def protected_rxn_idx(protected_rxn):
    #idx = []
    #for i in protected_rxn:
    #    idx.append(model.reactions.index(i))
    #return idx
#protected = protected_rxn_idx(protected_rxn)
#protected

In [8]:
GM, C, NC, P, Z, model_C = rank_reactions(model, G, U, confidence_scores, [], 2)   

/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/pymCADRE/rank/calc_expr_evidence.py:34: RuntimeWarning: All-NaN axis encountered
  U_GPR_min = [np.nanmin(l) for l in U_GPR]
/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/pymCADRE/rank/calc_expr_evidence.py:48: RuntimeWarning: All-NaN axis encountered
  E_X[i] = np.nanmax(tmp_lst)


Checking all reactions (FASTCC)...
Read LP format model from file /tmp/tmpczzh0sqg.lp
Reading time = 0.04 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Model consists of 602 blocked (inactive) reactions
Execution time of check_model_consistency: 42.490470443999996 s
check_model_consistency done ... 
Read LP format model from file /tmp/tmpf16c6edb.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros


/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/pymCADRE/rank/calc_conn_evidence.py:39: RuntimeWarning: invalid value encountered in true_divide
  I = A / rep  # define Influence Matrix


: 

: 

In [ ]:
for i in range(len(GM.metabolites)):
    if GM.metabolites[i].id == 'protein_c':
        GM.metabolites[i].formula = 'PROTEIN'

In [ ]:
GM.metabolites.protein_c

In [ ]:
try:
    
    print('Pruning reactions...')
    t0 = process_time()
    PM, cRes = prune_model(GM, P, C, Z, 1/3, '/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_XML_pymCADRE_INPUT/KeyMets_Recon3D_XML.txt', 1, 2)
    # Stop the stopwatch / counter
    t_stop = process_time()
    # compute elapsed time
    pruneTime = t_stop - t0
    
    print('PASS...', 'Function prune_model ran without error')

except (RuntimeError, TypeError, NameError) as inst:

    print('FAIL...', 'Function prune_model was terminated with the error:')
    print(type(inst))
    print(inst.args)

In [ ]:
io.write_sbml_model(model, '/home/jiang817/cybergut_project/Draft_model/Draft_model_with_pymCADRE/test.xml')